In [1]:
!wget "http://data.gdeltproject.org/gdeltv2/masterfilelist.txt"

--2025-03-26 18:05:44--  http://data.gdeltproject.org/gdeltv2/masterfilelist.txt
Resolving data.gdeltproject.org (data.gdeltproject.org)... 142.250.31.207, 142.251.167.207, 172.253.122.207, ...
Connecting to data.gdeltproject.org (data.gdeltproject.org)|142.250.31.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 111688014 (107M) [text/plain]
Saving to: ‘masterfilelist.txt’

100%[======================================>] 111,688,014 9.68MB/s   in 13s    

2025-03-26 18:05:59 (8.26 MB/s) - ‘masterfilelist.txt’ saved [111688014/111688014]



In [12]:

# !pip install newspaper3k
# !pip install --user --no-build-isolation newspaper3k

# !pip install --user --upgrade setuptools wheel

# !pip install --user --no-build-isolation feedfinder2

# !pip install --user newspaper3k


# !pip install lxml_html_clean

  Using cached feedfinder2-0.0.4.tar.gz (3.3 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [1 lines of output]
      ERROR: Can not execute `setup.py` since setuptools is not available in the build environment.
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [4]:
import pandas as pd
from datetime import datetime

import pandas as pd
import multiprocessing
import requests
import os
import re

import tldextract as tld
import urllib.parse as parse

from newspaper import Article
import zipfile
import multiprocessing


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/ihome/crc/install/python/ondemand-jupyter-python3.11-2023.09/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/ihome/crc/install/python/ondemand-jupyter-python3.11-2023.09/lib/python3.11/site-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/ihome/crc/install/python/ondemand-jupyter-python3.11-2023.09/l

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/ihome/crc/install/python/ondemand-jupyter-python3.11-2023.09/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/ihome/crc/install/python/ondemand-jupyter-python3.11-2023.09/lib/python3.11/site-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/ihome/crc/install/python/ondemand-jupyter-python3.11-2023.09/l

AttributeError: _ARRAY_API not found

ModuleNotFoundError: No module named 'newspaper'

In [ ]:
# parse files to pandas DF
links_df = pd.read_csv("masterfilelist.txt", sep=" ", header=None)

In [ ]:
links_df.columns = ["Col1", "Col2", "url"]

links_df.head()

In [ ]:
for row in links_df.itertuples():
    if '20221111031500.export.CSV.zip' in str(row.url):
        print(row)
        break

Pandas(Index=796521, Col1='3284', Col2='7a81f753d125cbc10acea058720c4dc8', url='http://data.gdeltproject.org/gdeltv2/20221111031500.export.CSV.zip')


In [ ]:
links_df.iloc[-3, :].url

'http://data.gdeltproject.org/gdeltv2/20250324234500.export.CSV.zip'

In [ ]:
def extract_date_from_url(url):
    try:
        date_segment = url.split("/")[-1]
        date_segment = date_segment.split(".")[0]
        #print(date_segment[:12])

        return datetime.strptime(date_segment[0:12], "%Y%m%d%H%M")

    except Exception as e:
        return None

In [ ]:
def extract_type_from_url(url):
    try:
        if "export" in url:
            return "export"
        elif "mentions" in url:
            return "mentions"
        elif "gkg" in url:
            return "gkg"
    except Exception as e:
        return ""

In [ ]:
# print(extract_date_from_url("http://data.gdeltproject.org/gdeltv2/20240419024500.gkg.csv.zip"))

In [ ]:
links_df["datetime"] = links_df["url"].apply(extract_date_from_url)
links_df["ftype"] = links_df["url"].apply(extract_type_from_url)

In [ ]:
# download file

In [ ]:
import os
from pathlib import Path
from urllib.request import urlretrieve
import requests

In [ ]:
def download_file(url, dateinfo):
    fname = url.split("/")[-1]
    datadirs = f"datadir_tambahan/{dateinfo.strftime('%Y/%m/%d')}"
#     print(datadirs)

    fpath = os.path.join(datadirs, fname)

    if not os.path.isdir(os.path.join(datadirs)):
        os.makedirs(os.path.join(datadirs))


    response = requests.get(url, stream=True)
    with open(fpath, "wb") as fout:
        for chunk in response.iter_content(chunk_size=10 * 1024):
            fout.write(chunk)



In [ ]:
# download_file("http://data.gdeltproject.org/gdeltv2/20240419030000.export.CSV.zip", datetime(2024, 1, 10))

In [ ]:
for row in links_df[
    (links_df["ftype"] == "export") &
    (links_df["datetime"] >= "2025-03-01") &
    (links_df["datetime"] < "2025-04-01")].itertuples():
    download_file(row.url, row.datetime)
#     break

In [ ]:
def urlStrip(url):

    try:
        temp = requests.head(url, allow_redirects=True, timeout=2).url
    except Exception as e:
        return None, None

    if temp[-1] != '/':
        temp += '/'
    ext = tld.extract(temp)
    parsed = parse.urlparse(temp)
    final_url = 'https://'
    if ext.subdomain == '':
        final_url += 'www.'
    final_url += (parsed.netloc + parsed.path)

    return (url, final_url.replace('/amp/', '/').replace('.amp', '').replace('/amphtml/', '/').strip('/'))

In [ ]:
def downloadArticle(url):

    article = Article(url, request_timeout=2, fetch_images=False)

    try:
        article.download()
        article.parse()

        content = article
        title = content.title
        images = content.images
        authors = content.authors
        text = content.text
        movies = content.movies
        date = content.publish_date
        tags = content.tags
        body = {
            'title': title,
#             'images': images,
            'authors': authors,
            'text': text,
#             'movies': movies,
            'date': date,
            'tags': tags,
            'article_url': url
        }


        return body
    except Exception:
        return None




In [ ]:
def job(url):
    url, strip_url = urlStrip(url)
    if url is not None:
        body = downloadArticle(strip_url)
        return [url, strip_url, body]
    else:
        return ['URL N/A', '', None]

In [ ]:
def process_file(file_path):
    df = pd.read_csv(file_path, header=None, sep="\t")
    df.rename(columns={60: "url"}, inplace=True)
    # filtered_df = df[df['url'].str.contains(r'trump', flags=re.IGNORECASE)]
    filtered_df = df
    filtered_df.drop_duplicates(subset="url", keep="first", inplace=True)
    urls = filtered_df['url'].tolist()

    # multiprocessing
    with multiprocessing.Pool(8) as pool:
        out = pool.map(job, urls)

    # Prepare the results for DataFrame insertion
    data = []
    for item in out:
        if item[2]:  # Check if the article was successfully crawled
            url, strip_url, body = item
            data.append([url, strip_url, body['title'], body['text'], body['date'], body['article_url'], body['authors'], body['tags']])

    # Convert the data into a DataFrame
    columns = ['original_url', 'strip_url', 'title', 'text', 'date', 'article_url', 'authors', 'tags']
    result_df = pd.DataFrame(data, columns=columns)

    # Merge the crawled data into the original DataFrame
    filtered_df = pd.merge(filtered_df, result_df, left_on='url', right_on='original_url', how='left')

    # Drop unnecessary columns
    filtered_df.drop(['original_url'], axis=1, inplace=True)

    # Save the result
    output_file = file_path.replace(".export.CSV", "_crawled_filtered.export.CSV")
    filtered_df.to_csv(output_file, index=False)

In [ ]:
homeDir = '/content/datadir_tambahan/2025/01'
for subfolder in os.listdir(homeDir):
  for filename in os.listdir(os.path.join(homeDir, subfolder)):
    if filename.endswith('.zip'):
      with zipfile.ZipFile(os.path.join(homeDir, subfolder, filename), 'r') as zip_ref:
        zip_ref.extractall('/content/parsed')
directory = '/content/parsed'

In [ ]:

# Iterate over files in the directory
for filename in os.listdir(directory):
    if filename.endswith(".export.CSV") and not filename.endswith("_crawled_filtered.export.CSV"):
        modified_filename = filename.replace(".export.CSV", "_crawled_filtered.export.CSV")
        if not os.path.exists(os.path.join(directory, modified_filename)):
            print(filename)
            file_path = os.path.join(directory, filename)
            process_file(file_path)
            #break

20250103083000.export.CSV
20250111004500.export.CSV
20250102134500.export.CSV
20250105193000.export.CSV
20250107033000.export.CSV
20250109043000.export.CSV
20250107100000.export.CSV
20250106141500.export.CSV
20250107043000.export.CSV
20250104221500.export.CSV
20250106180000.export.CSV
20250106020000.export.CSV
20250108144500.export.CSV
20250102034500.export.CSV
20250108210000.export.CSV
20250108200000.export.CSV
20250107123000.export.CSV
20250108073000.export.CSV
20250105214500.export.CSV
20250107001500.export.CSV
20250104224500.export.CSV
20250103044500.export.CSV
20250102151500.export.CSV
20250111031500.export.CSV
20250101113000.export.CSV
20250108223000.export.CSV
20250103051500.export.CSV
20250109074500.export.CSV
20250101161500.export.CSV


/usr/local/lib/python3.11/dist-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname EST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


20250108030000.export.CSV
20250103094500.export.CSV
20250111033000.export.CSV
20250103200000.export.CSV
20250111013000.export.CSV
20250106131500.export.CSV
20250104020000.export.CSV
20250107094500.export.CSV
20250103111500.export.CSV
20250105174500.export.CSV
20250106104500.export.CSV
20250102213000.export.CSV
20250108131500.export.CSV
20250107070000.export.CSV
20250103180000.export.CSV
20250110073000.export.CSV


/usr/local/lib/python3.11/dist-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname EST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


20250106213000.export.CSV
20250102030000.export.CSV
20250108110000.export.CSV


...

/usr/local/lib/python3.11/dist-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname EST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
Building prefix dict from /usr/local/lib/python3.11/dist-packages/jieba/dict.txt ...
DEBUG:jieba:Building prefix dict from /usr/local/lib/python3.11/dist-packages/jieba/dict.txt ...
Loading model from cache /tmp/jieba.cache
DEBUG:jieba:Loading model from cache /tmp/jieba.cache
Loading model cost 2.5740907192230225 seconds.
DEBUG:jieba:Loading model cost 2.5740907192230225 seconds.
Prefix dict has been built succesfully.
DEBUG:jieba:Prefix dict has been built succesfully.


20250110050000.export.CSV
20250102211500.export.CSV
20250101184500.export.CSV
20250110170000.export.CSV
20250110160000.export.CSV
20250104113000.export.CSV


/usr/local/lib/python3.11/dist-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname IST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


20250107143000.export.CSV
20250107101500.export.CSV
20250106004500.export.CSV
20250104170000.export.CSV
20250109084500.export.CSV
20250106193000.export.CSV
20250103190000.export.CSV
20250103134500.export.CSV
20250110030000.export.CSV
20250107201500.export.CSV
20250102180000.export.CSV
20250107063000.export.CSV
20250110213000.export.CSV
20250101140000.export.CSV
20250109164500.export.CSV
20250106164500.export.CSV
20250110084500.export.CSV
20250101121500.export.CSV
20250109171500.export.CSV
20250106054500.export.CSV
20250108180000.export.CSV
20250101211500.export.CSV


KeyboardInterrupt: 

In [ ]:
total_rows = pd.DataFrame(columns=['title', 'text', 'date', 'article_url'])
for filename in os.listdir('parsed'):
    if filename.endswith("_crawled_filtered.export.CSV"):
      rdf = pd.read_csv(os.path.join('parsed', filename))
      rsdf = rdf[['title', 'text', 'date', 'article_url']]
      total_rows = pd.concat([total_rows, rsdf], ignore_index=True)
total_rows.to_csv('combined_gdelt')


In [ ]:
# fdf = pd.read_csv('combined_gdelt.csv')
# fdf = fdf.dropna(subset=['date'])
fdf.head()

,Unnamed: 0,title,text,date,article_url
1,1,Driver of Tesla Cybertruck killed in explosion...,The driver of the Tesla Cybertruck who was kil...,2025-01-02 00:00:00,https://www.nypost.com/2025/01/02/us-news/las-...
3,3,Deadly attacks rattle US ahead of Trump inaugu...,A gruesome attack on revellers celebrating New...,2025-01-02 00:00:00,https://www.afr.com/world/north-america/deadly...
4,4,One dies as Tesla cybertruck explodes outside ...,"First responders, some wearing Hazmat gear, ga...",2025-01-02 08:16:26+00:00,https://www.punchng.com/one-dies-as-tesla-cybe...
5,5,Trump Has Promised to Build More Ships. He May...,Sign up for The Big Story newsletter to receiv...,2025-01-04 00:00:00,https://www.gilmermirror.com/2025/01/04/trump-...
7,7,Trump Loses It Over Flags Flying Half-Staff At...,LOADING ERROR LOADING\n\nPresident-elect Donal...,2025-01-04 09:17:11+00:00,https://www.huffingtonpost.co.uk/entry/trump-f...
